In [640]:
import pandas as pd
import math
import numpy as np

In [584]:
#importing targeted receivers and cb_wr dataset from dylan

targetedReceiver = pd.read_csv('/Users/alexmcgraw/Documents/Adv_Predictive_Modeling/APM-Term-Project/targetedReceiver.csv')
cb_wr = pd.read_csv('/Users/alexmcgraw/Documents/Adv_Predictive_Modeling/APM-Term-Project/cb_wr.csv')

mask = ((cb_wr['position'] == 'CB') | (cb_wr['position'] == 'WR') & (cb_wr['event'] == 'pass_forward') | (cb_wr['event']\
                                                                                                        == 'pass_arrived'))
#getting only the pass forward and arrived events for CB and WR
cb_wr_passes = cb_wr[mask]

#only one game
game1 = cb_wr_passes[(cb_wr_passes['gameId'] == 2018090600)]
game1 = game1[['time','playId', 'x', 'y', 'event','nflId','position','team']]

game1Receivers = targetedReceiver[(targetedReceiver['gameId'] == 2018090600)]


In [585]:
game1.head()

,time,playId,x,y,event,nflId,position,team
0,2018-09-07T01:07:18.099Z,75,81,45,pass_forward,2495454,WR,away
1,2018-09-07T01:07:18.099Z,75,83,35,pass_forward,2533040,WR,away
2,2018-09-07T01:07:18.099Z,75,80,20,pass_forward,2552689,CB,home
3,2018-09-07T01:07:18.099Z,75,77,45,pass_forward,2555383,CB,home
4,2018-09-07T01:07:19.200Z,75,81,48,pass_arrived,2495454,WR,away


In [586]:
game1Receivers.head()

,gameId,playId,targetNflId
0,2018090600,75,2495454
1,2018090600,146,2560854
2,2018090600,168,2543583
3,2018090600,190,2495454
4,2018090600,256,2543583


In [587]:
#Merging game and targeted receivers
game_receiver_merge = game1.merge(game1Receivers, how='outer', left_on = ['playId', 'nflId'], right_on = ['playId',
                                                                                                          'targetNflId'])
#replacing NaN with None
game_receiver_merge = game_receiver_merge.where(pd.notnull(game_receiver_merge), None)

#getting rid of instances where there is a WR that isn't the targeted one
game_receiver_merge = game_receiver_merge[(((game_receiver_merge['position'] == 'WR') & (game_receiver_merge['nflId'] == game_receiver_merge
                                                                 ['targetNflId'])) | \
                     (game_receiver_merge['position'] == 'CB'))]

#zipping x and y
game_receiver_merge['x,y'] = list(zip(game_receiver_merge['x'], game_receiver_merge['y']))

#dropping x and y columns
game_receiver_merge = game_receiver_merge.drop(['x', 'y'], axis=1)

#gets rid of scientific notation
pd.set_option('display.float_format', lambda x: '%.0f' % x) 

game_receiver_merge.head()

,time,playId,event,nflId,position,team,gameId,targetNflId,"x,y"
0,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
1,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)"
4,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)"
5,2018-09-07T01:07:19.200Z,75,pass_arrived,2552689,CB,home,None,None,"(77.11, 17.3)"
6,2018-09-07T01:07:18.099Z,75,pass_forward,2555383,CB,home,None,None,"(76.53, 44.93)"


In [588]:
# FIX SO WHEN ITERATING THRU EVERY GAME

#unique play ids
plays = pd.Series(game_receiver_merge['playId'].unique())

#creating a new DF, that only has plays that have both a WR and CB, bc sometimes plays have CB but no WR, bc none targeted
merge_fixed = pd.DataFrame(columns=game_receiver_merge.columns)

for play in plays:
    df_temp = game_receiver_merge[(game_receiver_merge['playId'] == play)]
    unique_pos = df_temp['position'].unique()
    if len(unique_pos) == 2:
        merge_fixed = merge_fixed.append(df_temp)
        
merge_fixed        


,time,playId,event,nflId,position,team,gameId,targetNflId,"x,y"
0,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
1,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)"
4,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)"
5,2018-09-07T01:07:19.200Z,75,pass_arrived,2552689,CB,home,None,None,"(77.11, 17.3)"
6,2018-09-07T01:07:18.099Z,75,pass_forward,2555383,CB,home,None,None,"(76.53, 44.93)"
7,2018-09-07T01:07:19.200Z,75,pass_arrived,2555383,CB,home,None,None,"(74.4, 47.11)"
10,2018-09-07T01:09:12.700Z,146,pass_forward,2552689,CB,home,None,None,"(38.1, 10.36)"
11,2018-09-07T01:09:13.200Z,146,pass_arrived,2552689,CB,home,None,None,"(37.4, 9.64)"
12,2018-09-07T01:09:12.700Z,146,pass_forward,2555383,CB,home,None,None,"(41.28, 38.93)"
13,2018-09-07T01:09:13.200Z,146,pass_arrived,2555383,CB,home,None,None,"(43.3, 37.96)"


In [589]:
#merging on itself
merged2_fixed = merge_fixed.merge(merge_fixed, on=['time', 'event'])
merged2_fixed.head()


,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y"
0,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
1,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2552689,CB,home,None,None,"(79.51, 20.0)"
2,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2555383,CB,home,None,None,"(76.53, 44.93)"
3,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
4,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)",75,2552689,CB,home,None,None,"(79.51, 20.0)"


In [590]:
#keeping rows where the player positions aren't the same (so no pairings between two WR or CB)
merged2_fixed = merged2_fixed[((merged2_fixed['position_x'] != merged2_fixed['position_y']))]
      
    
merged2_fixed.iloc[:10]


,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y"
1,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2552689,CB,home,None,None,"(79.51, 20.0)"
2,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2555383,CB,home,None,None,"(76.53, 44.93)"
3,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
6,2018-09-07T01:07:18.099Z,75,pass_forward,2555383,CB,home,None,None,"(76.53, 44.93)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
10,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",75,2552689,CB,home,None,None,"(77.11, 17.3)"
11,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",75,2555383,CB,home,None,None,"(74.4, 47.11)"
12,2018-09-07T01:07:19.200Z,75,pass_arrived,2552689,CB,home,None,None,"(77.11, 17.3)",75,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)"
15,2018-09-07T01:07:19.200Z,75,pass_arrived,2555383,CB,home,None,None,"(74.4, 47.11)",75,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)"
21,2018-09-07T01:09:12.700Z,146,pass_forward,2552689,CB,home,None,None,"(38.1, 10.36)",146,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)"
25,2018-09-07T01:09:12.700Z,146,pass_forward,2555383,CB,home,None,None,"(41.28, 38.93)",146,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)"


In [600]:
#Calculating the euclidean distance between the players
    
def euc_dist(item):
    return math.sqrt(((item['x,y_x'][0] - item['x,y_y'][0])**2) + 
                                     ((item['x,y_x'][1] - item['x,y_y'][1])**2))

merged2_fixed['Euc_dist'] = merged2_fixed.apply(euc_dist, axis=1)

"""for play in plays:
    df_temp = merged2_fixed[((merged2_fixed['playId_x'] == play) & (merged2_fixed['event'] == 'pass_forward'))]
    if len(df_temp) == 1:
        continue
    else:"""
        
    
merged2_fixed.head(35)


,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y",Euc_dist
1,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2552689,CB,home,None,None,"(79.51, 20.0)",25
2,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2555383,CB,home,None,None,"(76.53, 44.93)",4
3,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",25
6,2018-09-07T01:07:18.099Z,75,pass_forward,2555383,CB,home,None,None,"(76.53, 44.93)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",4
10,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",75,2552689,CB,home,None,None,"(77.11, 17.3)",31
11,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",75,2555383,CB,home,None,None,"(74.4, 47.11)",7
12,2018-09-07T01:07:19.200Z,75,pass_arrived,2552689,CB,home,None,None,"(77.11, 17.3)",75,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",31
15,2018-09-07T01:07:19.200Z,75,pass_arrived,2555383,CB,home,None,None,"(74.4, 47.11)",75,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",7
21,2018-09-07T01:09:12.700Z,146,pass_forward,2552689,CB,home,None,None,"(38.1, 10.36)",146,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)",28
25,2018-09-07T01:09:12.700Z,146,pass_forward,2555383,CB,home,None,None,"(41.28, 38.93)",146,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)",5


In [674]:
#cuts df by half. Keeps only rows where it is WR then CB, and not CB and WR (which is just a repeat)
merged2_fixed = merged2_fixed[(merged2_fixed['position_x'] == 'WR')]
print(merged2_fixed.shape)

for play in plays:
    df_temp = merged2_fixed[((merged2_fixed['playId_x'] == play))]
    #print(df_temp)
    if len(df_temp) > 1:
        
        ids = []
        idx = df_temp[(df_temp['event'] == 'pass_forward')]['Euc_dist'].idxmax()
        #for val in df_temp[df_temp['event'] == 'pass_forward']:
            #idx = df_temp['Euc_dist'].idxmax()
        #print(idx)
        nfl = df_temp['nflId_y'].loc[idx]
        ids = df_temp[df_temp['nflId_y'] == nfl].index.tolist()
        #print(ids)
        
        merged2_fixed.drop(ids, axis=0, inplace=True)
        

        #merged2_fixed.drop(df_temp['Euc_dist'].idxmax(), inplace=True)
        #merged2_fixed.drop()

print(merged2_fixed.shape)

(226, 17)
(144, 17)


In [579]:
#self merging AGAIN, so now have a row with the pass_forward and the pass_arrived. Much easier to calculate diff
ForArrStack = merged2_fixed.merge(merged2_fixed, on = ['playId_x', 'nflId_x', 'nflId_y'], suffixes=['_Xagain', '_Yagain'])
ForArrStack.columns


Index(['time_Xagain', 'playId_x', 'event_Xagain', 'nflId_x',
       'position_x_Xagain', 'team_x_Xagain', 'gameId_x_Xagain',
       'targetNflId_x_Xagain', 'x,y_x_Xagain', 'playId_y_Xagain', 'nflId_y',
       'position_y_Xagain', 'team_y_Xagain', 'gameId_y_Xagain',
       'targetNflId_y_Xagain', 'x,y_y_Xagain', 'Euc_dist_Xagain',
       'time_Yagain', 'event_Yagain', 'position_x_Yagain', 'team_x_Yagain',
       'gameId_x_Yagain', 'targetNflId_x_Yagain', 'x,y_x_Yagain',
       'playId_y_Yagain', 'position_y_Yagain', 'team_y_Yagain',
       'gameId_y_Yagain', 'targetNflId_y_Yagain', 'x,y_y_Yagain',
       'Euc_dist_Yagain'],
      dtype='object')

In [580]:
#Get rid of rows where the forward time is equal to or greater than the arrived time
ForArrStack = ForArrStack[(ForArrStack['time_Xagain'] < ForArrStack['time_Yagain'])]

In [581]:
#cut down on useless rows
print(ForArrStack.iloc[10])
ForArrStack.drop(['time_Yagain', 'targetNflId_y_Yagain', 'position_x_Yagain', 'position_y_Yagain', 'gameId_y_Yagain', 'gameId_x_Yagain', 'playId_y_Yagain', 'gameId_y_Xagain', 'targetNflId_x_Yagain'], axis=1, inplace=True)
print(ForArrStack.shape)

time_Xagain             2018-09-07T01:16:54.900Z
playId_x                                     320
event_Xagain                        pass_forward
nflId_x                                  2552600
position_x_Xagain                             WR
team_x_Xagain                               home
gameId_x_Xagain                       2018090600
targetNflId_x_Xagain                     2552600
x,y_x_Xagain                      (16.54, 41.86)
playId_y_Xagain                              320
nflId_y                                  2556445
position_y_Xagain                             CB
team_y_Xagain                               away
gameId_y_Xagain                             None
targetNflId_y_Xagain                        None
x,y_y_Xagain                       (19.9, 19.39)
Euc_dist_Xagain                               23
time_Yagain             2018-09-07T01:16:55.700Z
event_Yagain                        pass_arrived
position_x_Yagain                             WR
team_x_Yagain       

In [555]:
#Calc the Euclidean Difference from pass forward and pass arrived
ForArrStack['Euc_diff'] = ForArrStack['Euc_dist_Xagain'] - ForArrStack['Euc_dist_Yagain']
ForArrStack.head()

,time_Xagain,playId_x,event_Xagain,nflId_x,position_x_Xagain,team_x_Xagain,gameId_x_Xagain,targetNflId_x_Xagain,"x,y_x_Xagain",playId_y_Xagain,...,targetNflId_y_Xagain,"x,y_y_Xagain",Euc_dist_Xagain,event_Yagain,team_x_Yagain,"x,y_x_Yagain",team_y_Yagain,"x,y_y_Yagain",Euc_dist_Yagain,Euc_diff
1,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,...,None,"(79.51, 20.0)",25,pass_arrived,away,"(81.11, 47.87)",home,"(77.11, 17.3)",31,-6
5,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,...,None,"(76.53, 44.93)",4,pass_arrived,away,"(81.11, 47.87)",home,"(74.4, 47.11)",7,-3
9,2018-09-07T01:09:12.700Z,146,pass_forward,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)",146,...,None,"(38.1, 10.36)",28,pass_arrived,away,"(44.79, 36.79)",home,"(37.4, 9.64)",28,-1
13,2018-09-07T01:09:12.700Z,146,pass_forward,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)",146,...,None,"(41.28, 38.93)",5,pass_arrived,away,"(44.79, 36.79)",home,"(43.3, 37.96)",2,3
17,2018-09-07T01:09:12.700Z,146,pass_forward,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)",146,...,None,"(46.14, 25.29)",12,pass_arrived,away,"(44.79, 36.79)",home,"(45.65, 25.68)",11,0


In [569]:
#created a new DF and appended the CB nfl IDs and the Euclidean difference for each play
playerEucDiff = pd.DataFrame()
playerEucDiff['nflId_CB'] = ForArrStack['nflId_y']
playerEucDiff['Euc_diff'] = ForArrStack['Euc_diff']
playerEucDiff

,nflId_CB,Euc_diff
1,2552689,-6
5,2555383,-3
9,2552689,-1
13,2555383,3
17,2557958,0
21,2552689,9
25,2555383,-10
29,2557958,-2
33,2539334,-5
37,2539653,3


In [ ]:
#NOW. Need to go back and find which players, CBs, are closest to the targeted WR at the time of snap/pass forward,
#so only looking at that CB